# Dynamic Batch Size v.s. Static Batch Size

For OpenVINO dynamic shape usage, please refer to [Changing Input Shapes](https://docs.openvino.ai/2023.2/openvino_docs_OV_UG_ShapeInference.html) 

#### Load OpenVINO models

In [ ]:
import time

import numpy as np
from ipywidgets import interact, widgets
from openvino import Core

model_name = "resnet50"
model_type = "int8"

models = ["resnet18", "resnet34", "resnet50", "resnet101", "resnet152"]


@interact(option=widgets.Dropdown(options=models, value=model_name, description="model name"))
def read_dropdown(option):
    global model_name
    model_name = option


model_types = ["fp32", "fp16", "int8"]


@interact(option=widgets.Dropdown(options=model_types, value=model_type, description="model type"))
def read_dropdown(option):
    global model_type
    model_type = option


In [ ]:
def set_batch_size(model, batch_size: int):
    shapes = {}
    for input_layer in model.inputs:
        shapes[input_layer] = input_layer.partial_shape
        shapes[input_layer][0] = batch_size
    model.reshape(shapes)


core = Core()
model = core.read_model(f"models/{model_name}/{model_type}/model.xml")

#### Set batch size

In [ ]:
set_batch_size(model, -1)
dynamic_model = core.compile_model(model, "CPU")
dynamic_model.input().get_partial_shape()

In [ ]:
set_batch_size(model, 1)
static_model_b1 = core.compile_model(model, "CPU")
static_model_b1.input().get_partial_shape()

In [ ]:
set_batch_size(model, 8)
static_model_b8 = core.compile_model(model, "CPU")
static_model_b8.input().get_partial_shape()

#### Benchmark Models

In [ ]:
SEC = 10


def benchmark_model(model, batch_size: int):
    inputs = np.random.rand(batch_size, 3, 224, 224)
    
    count = 0
    start = time.time()
    while time.time() - start < SEC:
        model(inputs)
        count += batch_size
        
    print(f"FPS={count / SEC:.2f}")

1. Dynamic shape model with batch size = 1

In [ ]:
benchmark_model(dynamic_model, 1)

2. Static shape model with batch size = 1

In [ ]:
benchmark_model(static_model_b1, 1)

3. Dynamic shape model with batch size = 8

In [ ]:
benchmark_model(dynamic_model, 8)

4. Static shape model with batch size = 8

In [ ]:
benchmark_model(static_model_b8, 8)